# Análisis de tópicos

En esta notebook vamos a descomponer en tópicos un conjunto de textos. La descomposición en tópicos pueden pensarla a la vez como un proceso de reducción dimensional, al describir los datos en el espacio de tópicos en vez del espacio original de features), o como un proceso de *clustering*, al agrupar los textos en dichos tópicos.

Vamos a aplicar dos algoritmos (NMF y LDA) sobre el dataset de letras de tango visto en clases anteriores.

In [77]:
#Importamos las librerías a utilizar 
import requests # Realizar peticiones al servidor
from bs4 import BeautifulSoup as BS #Extrae, proporciona métodos y herramientas para procesar páginas web de manera más sencilla
import pickle
import pandas as pd
import numpy as np 
pd.set_option('max_colwidth',150) # para que las columnas del dataframe muestren hasta 150 caracteres

In [78]:
url='https://www.literatura.us/garciamarquez/'
 # Hacemos la petición HTTP
response=requests.get(url)
    
# soup es un objeto manipulable creado a partir del contenido de la página y BeautifulSoup
soup = BS(response.content)

# Extraemos el elemento p de la  clase = "MsoNormal" dónde se encuentra el cuento 


In [85]:
import re
# Encontrar todos los elementos <p> con la clase 'MsoNormal'
paragraphs = soup.find_all(class_='MsoNormal')

In [80]:
titulos = []
enlaces = []
anios   = ['1947', '1949', '1949', '1950', '1950', '1955', '1954', '1961', '1962','1962','1962','1962',
           '1962','1962','1962', '1962',
          '1968', '1968', '1970', '1972', '1975', '1981', '1979', '1981', 
           '1982', '1980', '1978', '1980', '1979', '1980', '1982', '1978', '1978', '1976','1967','1967'
           ,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

for p in paragraphs:
    # Buscar solo los elementos <a> directamente dentro del elemento <p>
    a_tags = p.find_all('a')
    texto_p =p.text.strip()
    for a in a_tags:
        titulos.append(a.text.strip())
        enlaces.append(a['href'])
    

cuentos={
         'titulos': titulos,
         'enlaces': enlaces,
         'años' : anios
}

cuentos=pd.DataFrame(cuentos)

cuentos.dropna(inplace=True)

In [82]:
text=[]
for url in cuentos.enlaces:
    # Hacemos la petición HTTP
    response=requests.get(url)
    
    # soup es un objeto manipulable creado a partir del contenido de la página y BeautifulSoup
    soup = BS(response.content)

    # Extraemos el elemento p de la  clase = "MsoNormal" dónde se encuentra el cuento 
    paragraph= soup.find(attrs = {"class": "MsoNormal"}).text

    # Palabras a quitar del párrafo
    palabras_remplazar='Literatura\n.us\nMapa de la biblioteca | Aviso Legal | Quiénes Somos | Contactar\n      \xa0\n\n\n\n'

    # Quitamos palabras de publicidad que están en párrafo 
    paragraphs = paragraph.replace(palabras_remplazar, "")
    
    # Iteramos para todo párrafo y vemos el texto contenido
    # Podemos hacer una lista y concatenar todo con un salto de línea
    # paragraphs= '\n \n'.join(paragraphs)

    # Adicionamos cada texto al diccionario texts, y lo relacionamos con su título 
    text.append(paragraphs)
cuentos['textos']=text

In [83]:
import re       # librería de expresiones regulares
import string   # librería de cadena de caracteres
# Defino una función que recibe un texto y devuelve el mismo texto sin singnos,
def clean_text_round1(text):
    # pasa las mayúsculas del texto a minúsculas
    text = text.lower() 

    # reemplaza texto entre corchetes por espacio en blanco.. ¿ y \% no se..
    text = re.sub(r'\[.*?¿\]\%', ' ', text)

    #Eliminar tanto los espacios en blanco no rompibles ("\xa0"), saltos de línea y carácter "\xad"
    text=re.sub(r'[\xa0\n¿!]', '', text) 

    #Eliminar el carácter "\xad"
    text=re.sub(r'[\xad]', '', text) 

    #Eliminar los retornos de carro ("\r") 
    text=re.sub(r'[\r]', ' ', text)  

    # Eliminar guiones (-), en dash (–) y em dash (—)
    text = re.sub(r'[-\u2013\u2014]', '', text) 

    text=re.sub(r'<(?!\/?[a-zA-Z])', '', text)

    # reemplaza singnos de puntuación por espacio en blanco.. %s -> \S+ es cualquier caracter que no sea un espacio en blanco
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 

    # Sacamos comillas, los puntos suspensivos, <<, >>
    text = re.sub('[‘’“”…«»]', '', text)

    # remueve palabras que contienen números.
    text = re.sub('\w*\d\w*', ' ', text)        

    return text

# Defino una función anónima que al pasarle un argumento devuelve el resultado de aplicarle la función anterior a este mismo argumento
round1 = lambda x: clean_text_round1(x)

# Dataframe que resulta de aplicarle a las columnas la función de limpieza
# Aplicar la limpieza a las columnas deseadas
cuentos['textos'] = cuentos['textos'].apply(clean_text_round1)
cuentos['titulos'] = cuentos['titulos'].apply(clean_text_round1)

#Lo pickleamos y guardamos 
cuentos.to_pickle(r"C:\Users\HP\Documents\Ciencia De Datos\Análisis de los funerales de la mamá grande\corpus.pkl")

In [86]:
# Importamos nltk para extraer stopwords 
import nltk 
nltk.download('stopwords')

# Librería para hacer wordclouds
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
# Objetos de sklearn para hacer tópicos
from sklearn.feature_extraction.text import CountVectorizer # Contador de frecuencia
from sklearn.feature_extraction.text import TfidfTransformer # Creador de tf-idf

# Algoritmos de descomposición de tópicos
from sklearn.decomposition import NMF 
from sklearn.decomposition import LatentDirichletAllocation

### Construcción de la matriz documentos-términos

Vamos a construir esta matriz con valorización tf-idf, es decir, además de la frecuencia del término vamos a ponderar la especificidad. 
Esto lo hacemos en dos pasos: primero, describimos nuestros datos mediante frecuencia de términos; luego, le agregamos la valorización de la especificidad. 
Vamos además a remover las *stopwords* obtenidas de *nltk*.

In [ ]:
# Lista de stopwords
stopwords = nltk.corpus.stopwords.words('spanish')

# Creamos el objeto contador de palabras, pidiéndole que remueve
# las stopwords, los términos que aparecen en un único documento (min_df)
# y los términos que aparecen en más del 70% de los documentos (max_df).
# Esto es para eliminar palabras raras (o errores de tipeo) y 
# términos que seguramente son stopwords no incluídos en la lista
count = CountVectorizer(min_df = 2, max_df = 0.70, stop_words = stopwords)

# Ajustamos con los datos. Acá especificamente creamos una matriz documentos-términos
x_count = count.fit_transform(df['letra'])

# Dimensions de la matriz doc-tér
print(x_count.shape)

Podemos tranquilamente trabajar sobre la matriz de documentos descritos a través de la frecuencia de las palabras o bien, como es habitual, ponderar la especificidad de los términos mediante tf-idf. Esto lo hacemos de la siguiente manera:

In [ ]:
# Creamos el objeto tf-idf. Le decimos además que devuelva los
# vectores documento con norma euclídea igual a 1 (norm = 'l2')
tfidf = TfidfTransformer(norm = 'l2')

# Creamos la matriz tf-idf a partir de la matriz de frecuencias
x_tfidf = tfidf.fit_transform(x_count)

### Aplicación de NMF sobre el corpus

Vamos a buscar los tópicos en nuestro corpus de textos a través de la descomposición en matrices no-negativas. Elijamos por ejemplo 5 tópicos:

In [ ]:
# Elijamos la cantidad de tópicos
n_components = 5

# Construímos el objeto NMF con los tópicos indicados 
nmf = NMF(n_components = n_components)

# Aplicamos sobre nuestros datos
x_nmf = nmf.fit_transform(x_tfidf)

# Dimensión de la matriz transformada
print(x_nmf.shape)

Veamos qué significa cada tópico. Lo que hay que indentificar es cuáles son los índices con mayor peso en cada componente y a qué término le corresponde. Para ello primero vamos a invertir el diccionario de vocabulario (para obtener otro del estilo "índice: término") y ordenar los índices de mayor a menor en cada componente: